# Generate Consistent Images for the CNN

*Based on the code Chris Birchall used to [find Wally](https://tech.ovoenergy.com/cheating-at-wheres-wally/).*

In [ ]:
import cv2
import glob
import os.path

In [ ]:
# this sort-of works, however, it stalls out at the end of the image...
def multiply_image(name):
    img = cv2.imread("../data/test/{}.jpg".format(name))
    for y in range(0, 10000, 500):
        start_y = y
        end_y = y + 1000 
        for x in range(0, 7600, 500):
            start_x = x
            end_x = (x+500) + 1000
            cropped_img = img[start_y:end_y, start_x:end_x]
            cv2.imwrite("../data/cropped_images/{}-{}-{}.png".format(name, y, x), cropped_img)
    
for path in glob.glob("../data/test/*.jpg"):
    name = os.path.splitext(os.path.basename(path))[0]
    multiply_image(name)